In [1]:
# Python3
import urllib.request
import sqlite3
from sqlite3 import Error
from bs4 import BeautifulSoup, Tag

database = "./ACCT/database.db"

headTblFields = """accNum text NOT NULL,
headId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
documentType text NOT NULL,
publicDocCount integer DEFAULT 0,
periodOfReport text,
filedDate text,
changedDate text,
schemaVersion text,
dateOfOriginalSubmission text,
notSubjectToSection16 integer DEFAULT 0,
issuerName text,
issuerCik text,
issuerIndustrialClassification text,
issuerIrs text,
issuerIncorpState text,
issuerFiscalYrEnd text,
issuerBusinessStreet1 text,
issuerBusinessCity text,
issuerBusinessState text,
issuerBusinessZip text,
issuerBusinessPhone text,
issuerMailStreet1 text,
issuerMailStreet2 text,
issuerMailCity text,
issuerMailState text,
issuerMailZip text,
issuerTradingSymbol text,
rptOwnerName text,
rptOwnerCik text DEFAULT 0,
rptOwnerFormType text,
rptOwnerSecAct text,
rptOwnerSecFileNum text,
rptOwnerFilmNum text,
rptOwnerBusinessPhone text,
rptOwnerStreet1 text,
rptOwnerStreet2 text,
rptOwnerCity text,
rptOwnerState text,
rptOwnerZipCode text,
rptOwnerStateDescription text,
rptOwnerisDirector integer DEFAULT 0,
rptOwnerisOfficer integer DEFAULT 0,
rptOwnerisTenPercentOwner integer DEFAULT 0,
rptOwnerisOther integer DEFAULT 0"""

dTTblFields = """accNum text NOT NULL,
dTId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
securityTitle text,
conversionOrExercisePrice text,
transactionDate text,
transactionFormType text,
transactionCode text,
equitySwapInvolved text,
transactionShares text,
transactionPricePerShare text,
transactionAcquiredDisposedCode text,
exerciseDate text,
expirationDate text,
underlyingSecurityTitle text,
underlyingSecurityShares text,
sharesOwnedFollowingTransaction text,
directOrIndirectOwnership text,
footNoteId text,"""

nDTTblFields = """accNum text NOT NULL,
nDTId text PRIMARY KEY NOT NULL,
rowNumber integer DEFAULT 1,
securityTitle text,
transactionDate text,
transactionFormType text,
transactionCode text,
equitySwapInvolved text,
transactionTimelines text,
transactionShares text,
transactionPricePerShare text,
transactionAcquiredDisposedCode text,
sharesOwnedFollowingTransaction text,
directOrIndirectOwnership text,
natureOfOwnership text,
footNoteId text,"""

footNoteTblFields = """accNum text NOT NULL,
rowNumber integer DEFAULT 1,
footNoteId text PRIMARY KEY NOT NULL,
fId text,
originalTableType text,
footNoteField text,
footNote text"""

dTTblSql = dTTblFields + "FOREIGN KEY(footNoteId) REFERENCES form4footNote(footNoteId)"
nDTTblSql = nDTTblFields + "FOREIGN KEY(footNoteId) REFERENCES form4footNote(footNoteId)"

In [2]:
textHeadMap = {"ACCESSION NUMBER": "accNum",
              "CONFORMED SUBMISSION TYPE": "documentType",
              "PUBLIC DOCUMENT COUNT": "publicDocCount",
              "CONFORMED PERIOD OF REPORT": "periodOfReport",
              "FILED AS OF DATE": "filedDate",
              "DATE AS OF CHANGE": "changedDate"}
issuerMap = {"COMPANY CONFORMED NAME": "issuerName",
              "CENTRAL INDEX KEY": "issuerCik",
              "STANDARD INDUSTRIAL CLASSIFICATION": "issuerIndustrialClassification",
              "IRS NUMBER": "issuerIrs",
              "STATE OF INCORPORATION": "issuerIncorpState",
              "FISCAL YEAR END": "issuerFiscalYrEnd"}
issuerBizMap = {"STREET 1": "issuerBusinessStreet1",
              "CITY": "issuerBusinessCity",
              "STATE": "issuerBusinessState",
              "ZIP": "issuerBusinessZip",
              "BUSINESS PHONE": "issuerBusinessPhone"}
issuerMailMap = {"STREET 1": "issuerMailStreet1",
              "STREET 2": "issuerMailStreet2",
              "CITY": "issuerMailCity",
              "STATE": "issuerMailState",
              "ZIP": "issuerMailZip"}
ownerMap = {"COMPANY CONFORMED NAME": "rptOwnerName",
          "CENTRAL INDEX KEY": "rptOwnerCik",
          "FORM TYPE": "rptOwnerFormType",
          "SEC ACT": "rptOwnerSecAct",
          "SEC FILE NUMBER": "rptOwnerSecFileNum",
          "FILM NUMBER": "rptOwnerFilmNum",
          "STREET 1": "rptOwnerStreet1",
          "STREET 2": "rptOwnerStreet2",
          "CITY": "rptOwnerCity",
          "STATE": "rptOwnerState",
          "ZIP": "rptOwnerZipCode",
          "BUSINESS PHONE": "rptOwnerBusinessPhone"}
xml2SqlOwnFields = {"isDirector" : "rptOwnerisDirector",
                 "isOfficer": "rptOwnerisOfficer", 
                 "isTenPercentOwner": "rptOwnerisTenPercentOwner", 
                 "isOther": "rptOwnerisOther"}
xmlEtcHeaders = ["schemaVersion", "documentType", "periodOfReport", "dateOfOriginalSubmission", "notSubjectToSection16"]
xmlIssHeaders = ["issuerCik", "issuerName", "issuerTradingSymbol"]
xmlOwnHeaders = ["rptOwnerCik", "rptOwnerName", "rptOwnerStreet1", "rptOwnerStreet2", "rptOwnerCity", "rptOwnerState","rptOwnerZipCode", "rptOwnerStateDescription", "isDirector", "isOfficer", "isTenPercentOwner", "isOther"]

In [25]:
def parseSecHeader(lines, sqlMap, sqlDic):
    lines = lines.replace(":", "").split('\n')
    for line in lines:
        if "\t" in line:
            parts = line.split('\t')
            parts = [x for x in parts if x != '']
            if parts and parts[0] in sqlMap:
                sqlDic[sqlMap[parts[0]]] = parts[1]
    return sqlDic

def addXmlHeader(sqlDic, xmlFields, xml2SqlMap):
    for field in xmlFields:
        val = soup.find(field)
        if val:
            val = val.getText()
        key = field
        if xml2SqlMap and field in xml2SqlMap:
            key = xml2SqlMap[field]
        if key == 'dateOfOriginalSubmission' and val:
            val = val.replace("-", "")
        sqlDic[key] = val
        
def addXmlHeaderMultiple(sqlDics, xmlFields, xml2SqlMap, index):
    for field in xmlFields:
        key = field
        if xml2SqlMap and field in xml2SqlMap:
            key = xml2SqlMap[field]
        
        vals = soup.find_all(field)
        if vals:
            sqlDics[key] = vals[index].getText()
#         for i in range(len(sqlDics)):
#             sqlDics[i][key] = vals[i].getText()

def parseXml(xmlFieldList, transactionsList, transactionType, accNum):
    xmlLis = xmlFieldList.split('\n')
    xmlFields = []
    for field in xmlLis:
        fieldParts = field.split(' ')
        xmlFields.append(fieldParts[0])

    sqlDic = {}
    if transactionType == 'derivativeTransaction':
        footNoteTag = 'dt'
    else:
        footNoteTag = 'ndt'
    for row in range(len(transactionsList)):
        sqlDic[row+1] = {}
        sqlDic[row+1]['footnotes'] = []
        for field in xmlFields:
            if field == 'accNum':
                val = accNum
            elif field == 'rowNumber':
                val = row+1
            elif field == 'dTId' or field == 'nDTId':
                val = accNum+'-'+str(row+1)
            elif field == 'footNoteId':
                continue
            elif field == 'transactionTimelines':
                field = 'transactionTimeliness'
            else:
                results = transactionsList[row].find(field)
                if results:
                    val = ''.join([str(x) for x in results.contents if x != '\n'])
                    for x in results.contents:
                        if x and isinstance(x, Tag):
                            fId = x.get('id')
                            if fId is not None:
                                foot = {'accNum': accNum,
                                       'rowNumber': row+1,
                                       'footNoteId': accNum+'-'+ str(row+1)+'-'
                                        +footNoteTag + '-' + fId + '-' + field,
                                       'fId': fId,
                                       'originalTableType': transactionType,
                                       'footNoteField': field
                                       }
                                sqlDic[row+1]['footnotes'].append(foot)
                else:
                    val = None
            if field == 'transactionTimeliness':
                field = 'transactionTimelines'
            sqlDic[row+1][field] = val
    return sqlDic

def fillFootNoteText(footnotes):
    for note in footnotes:
        fId = note['fId']
        footnote = soup.find('footnote', {'id': fId})
        if footnote:
            note['footNote'] = footnote.contents[0].replace('\n', '')

In [24]:
url = "https://www.sec.gov/Archives/edgar/data/1084869/000108486913000040/0001084869-13-000040.txt"
response = urllib.request.urlopen(url)
the_page = response.read()
content = the_page.decode(encoding='latin-1')
begin = content.find("<SEC-DOCUMENT>")
end = content.find("</SEC-DOCUMENT>")#content.find("-----END")
xmlFile = content[begin:end]
soup = BeautifulSoup(xmlFile, 'xml')

accNum = '1084869/000108486913000040/0001084869-13-000040'
nonDerivativeTransaction = soup.find_all("nonDerivativeTransaction")
sql = parseXml(dTTblFields, nonDerivativeTransaction, 'nonDerivativeTransaction', accNum)
sql

{1: {'accNum': '1084869/000108486913000040/0001084869-13-000040',
  'conversionOrExercisePrice': None,
  'dTId': '1084869/000108486913000040/0001084869-13-000040-1',
  'directOrIndirectOwnership': '<value>I</value>',
  'equitySwapInvolved': '0',
  'exerciseDate': None,
  'expirationDate': None,
  'footnotes': [{'accNum': '1084869/000108486913000040/0001084869-13-000040',
    'fId': 'F1',
    'footNoteField': 'transactionShares',
    'footNoteId': '1084869/000108486913000040/0001084869-13-000040-1-ndt-F1-transactionShares',
    'originalTableType': 'nonDerivativeTransaction',
    'rowNumber': 1},
   {'accNum': '1084869/000108486913000040/0001084869-13-000040',
    'fId': 'F2',
    'footNoteField': 'transactionPricePerShare',
    'footNoteId': '1084869/000108486913000040/0001084869-13-000040-1-ndt-F2-transactionPricePerShare',
    'originalTableType': 'nonDerivativeTransaction',
    'rowNumber': 1}],
  'rowNumber': 1,
  'securityTitle': '<value>Class A Common Stock</value>',
  'sharesOwn

In [26]:
### Database utility functions ###
def connectToDb(db):
    try:
        conn = sqlite3.connect(db)
        return conn
    except Error as e:
        print(e)

def createTable(conn, tableName, tableFields):
    try:
        c = conn.cursor()
        create_sql = "CREATE TABLE IF NOT EXISTS "+tableName+" (" + tableFields +");"
        c.execute(create_sql)
    except Error as e:
        print(e)
        
def insertToTable(table, dictionary, conn):
    columns = ', '.join(dictionary.keys())
    placeholders = ', '.join('?' * len(dictionary))
    sql = 'INSERT INTO ' + table + ' ({}) VALUES ({})'.format(columns, placeholders)
    try:
        conn.execute(sql, list(dictionary.values()))
    except Exception as e:
        print(e)
        conn.close()
        return
    conn.commit()
    
def insertToTransacTables(sql, tableName):
    for rowNum in sql.keys():
        row = sql[rowNum]
        footnotes = row.pop('footnotes')
        fillFootNoteText(footnotes)

        conn = connectToDb(database)
        insertToTable(tableName, row, conn)

        for note in footnotes:
            insertToTable('form4footNote', note, conn)

#def executeQuery(sql, )

In [27]:
def parseHead(soup, accNum):
    sqlDic = {} 

    # SEC-HEADER
    sec_header = soup.find("ACCEPTANCE-DATETIME").getText()
    issuerBegin = sec_header.find("ISSUER")
    ownerBegin = sec_header.find("REPORTING-OWNER")

    # Process top of SEC-HEADER
    etcHeader = sec_header[:issuerBegin]
    sqlDic = parseSecHeader(etcHeader, textHeadMap, sqlDic)
    addXmlHeader(sqlDic, xmlEtcHeaders, None)

    # SEC-HEADER Issuer section
    issuerHead = sec_header[issuerBegin:ownerBegin]

    # Issuer company data
    sqlDic = parseSecHeader(issuerHead, issuerMap, sqlDic)
    addXmlHeader(sqlDic, xmlIssHeaders, None)

    bizBegin = issuerHead.find("BUSINESS ADDRESS")
    mailBegin = issuerHead.find("MAIL ADDRESS")

    # SEC-HEADER Issuer business address
    issuerHeadBiz = issuerHead[bizBegin:mailBegin]
    sqlDic = parseSecHeader(issuerHeadBiz, issuerBizMap, sqlDic)

    # SEC-HEADER Issuer mail address
    issuerHeadMail = issuerHead[mailBegin:]
    sqlDic = parseSecHeader(issuerHeadMail, issuerMailMap, sqlDic)

    # SEC-HEADER Reporting owners section & insert into table
    rOwners = sec_header.split("REPORTING-OWNER:")[1:]
    for index in range(len(rOwners)):
        sqlDic = parseSecHeader(rOwners[index], ownerMap, sqlDic)
        addXmlHeaderMultiple(sqlDic, xmlOwnHeaders, xml2SqlOwnFields, index)
        sqlDic['rowNumber'] = index+1 
        sqlDic['accNum'] = accNum
        sqlDic['headId'] = accNum + "-" + str(index+1)
        insertToTable('form4head', sqlDic, conn)

def parseTransacs(soup, accNum):
    # Process derivative transactions
    derivativeTransactions = soup.find_all("derivativeTransaction")
    sql = parseXml(dTTblFields, derivativeTransactions, 'derivativeTransaction', accNum)
    insertToTransacTables(sql, 'form4dT')

    # Process non-derivative transactions
    nonDerivativeTransactions = soup.find_all("nonDerivativeTransaction")
    sqlNT = parseXml(nDTTblFields, nonDerivativeTransactions, 'nonDerivativeTransaction', accNum)
    insertToTransacTables(sqlNT, 'form4nDT')

In [29]:
conn = connectToDb(database)
createTable(conn, 'form4Head', headTblFields)
createTable(conn, 'form4dT', dTTblSql)
createTable(conn, 'form4nDT', nDTTblSql)
createTable(conn, 'form4footNote', footNoteTblFields)

#fname = "2007_4A_accNum"
#fname = "2007Form4.csv"
fname = "flowers_com_inc2014.txt"
with open(fname) as f:
    urls = f.read().splitlines()

thisTime = urls
for link in thisTime:
    #link = "0001084869-07-000035"
    url = "https://www.sec.gov/Archives/" + link
    #url = "https://www.sec.gov/Archives/edgar/data/1214101/0001104659-07-084171.txt"
    response = urllib.request.urlopen(url)
    the_page = response.read()
    content = the_page.decode(encoding='latin-1')
    file = open("test", "w")
    file.write(content) 

    parts = link.split('/')
    if len(parts) == 5: #.../data/1084869/000108486914000025/0001084869-14-000025.txt
        accNum = parts[len(parts)-3] + '/' +parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]
    if len(parts) == 4: #.../data/1214101/0001104659-07-084171.txt
        accNum = parts[len(parts)-2] + '/' + parts[len(parts)-1].split('.')[0]

    # start parsing
    begin = content.find("<SEC-DOCUMENT>")
    end = content.find("-----END")
    xmlFile = content[begin:end]
    soup = BeautifulSoup(xmlFile, 'xml')

    parseHead(soup, accNum)
    parseTransacs(soup, accNum)

In [72]:
ignoredFields = ['accNum', 'rowNumber', 'documentType', 'filedDate', 'changedDate', 'periodOfReport', 'dateOfOriginalSubmission', "headId", 'rptOwnerFormType', 'rptOwnerSecAct', 'rptOwnerSecFileNum', 'rptOwnerFilmNum', 'rptOwnerBusinessPhone', 'rptOwnerStateDescription']
headFields = headTblFields.replace("\n", "").split(",")
headFields = [field.split(" ")[0] for field in headFields]
headFields = [field for field in headFields if field not in ignoredFields]
headFields

['publicDocCount',
 'schemaVersion',
 'notSubjectToSection16',
 'issuerName',
 'issuerCik',
 'issuerIndustrialClassification',
 'issuerIrs',
 'issuerIncorpState',
 'issuerFiscalYrEnd',
 'issuerBusinessStreet1',
 'issuerBusinessCity',
 'issuerBusinessState',
 'issuerBusinessZip',
 'issuerBusinessPhone',
 'issuerMailStreet1',
 'issuerMailStreet2',
 'issuerMailCity',
 'issuerMailState',
 'issuerMailZip',
 'issuerTradingSymbol',
 'rptOwnerName',
 'rptOwnerCik',
 'rptOwnerStreet1',
 'rptOwnerStreet2',
 'rptOwnerCity',
 'rptOwnerState',
 'rptOwnerZipCode',
 'rptOwnerisDirector',
 'rptOwnerisOfficer',
 'rptOwnerisTenPercentOwner',
 'rptOwnerisOther']

In [73]:
query = "select A.accNum as aAcc, B.accNum as bAcc from form4head A, form4head B where A.documentType = '4/A' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate"
for field in headFields:
    query += " and A." + field + " = " "B." + field
query += ";"
print(query)

cur = conn.cursor()
cur.execute(query)
matches = cur.fetchall()
matches

select A.accNum as aAcc, B.accNum as bAcc from form4head A, form4head B where A.documentType = '4/A' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate and A.publicDocCount = B.publicDocCount and A.schemaVersion = B.schemaVersion and A.notSubjectToSection16 = B.notSubjectToSection16 and A.issuerName = B.issuerName and A.issuerCik = B.issuerCik and A.issuerIndustrialClassification = B.issuerIndustrialClassification and A.issuerIrs = B.issuerIrs and A.issuerIncorpState = B.issuerIncorpState and A.issuerFiscalYrEnd = B.issuerFiscalYrEnd and A.issuerBusinessStreet1 = B.issuerBusinessStreet1 and A.issuerBusinessCity = B.issuerBusinessCity and A.issuerBusinessState = B.issuerBusinessState and A.issuerBusinessZip = B.issuerBusinessZip and A.issuerBusinessPhone = B.issuerBusinessPhone and A.issuerMailStreet1 = B.issuerMailStreet1 and A.issuerMailStreet2 = B.issuerMailStreet2 and A.issuerMailCity = B.issuerMailCity and A.issuerMailState = B.issuerMailState and A.issuerMailZip

[('1084869/000108486914000025/0001084869-14-000025',
  '1084869/000108486914000024/0001084869-14-000024'),
 ('1084869/000114036114044053/0001140361-14-044053',
  '1084869/000114036114043945/0001140361-14-043945'),
 ('1084869/000114036114044052/0001140361-14-044052',
  '1084869/000114036114043865/0001140361-14-043865'),
 ('1084869/000108486914000016/0001084869-14-000016',
  '1084869/000108486914000014/0001084869-14-000014'),
 ('1084869/000108486914000015/0001084869-14-000015',
  '1084869/000108486914000010/0001084869-14-000010')]

In [ ]:
for match in matches:
    aDT = cur.execute("select * from form4dT where accNum = '" + match[0]+ + "';").fetchall()
    bDT = cur.execute("select * from form4dT where accNum = '" + match[0]+ + "';").fetchall()
    aNDT = cur.execute("select * from form4ndT where accNum = '" + match[0]+ + "';").fetchall()
    bNDT = cur.execute("select * from form4ndT where accNum = '" + match[0]+ + "';").fetchall()
    
    break

In [17]:
def main():
    conn = connectToDb(database)
    createTable(conn, 'form4Head', headTblFields)
    createTable(conn, 'form4dT', dTTblSql)
    createTable(conn, 'form4nDT', nDTTblSql)
    createTable(conn, 'form4footNote', footNoteTblFields)
    
    fname = "2007_4A_accNum"
    with open(fname) as f:
        urls = f.read().splitlines()
        
    int i = 0
    for link in urls:
        url = "https://www.sec.gov/Archives/" + link
        url = "https://www.sec.gov/Archives/edgar/data/1214101/0001104659-07-084171.txt"
        response = urllib.request.urlopen(url)
        the_page = response.read()
        content = the_page.decode(encoding='latin-1')
        file = open("test", "w")
        file.write(content) 

        parts = urls[1].split('/')
        accNum = parts[len(parts)-1].split('.')[0]
        #accNum = '0001104659-07-084171'

        # start parsing
        begin = content.find("<SEC-DOCUMENT>")
        end = content.find("-----END")
        xmlFile = content[begin:end]
        soup = BeautifulSoup(xmlFile, 'xml')
        
        parseHead(soup)
        parseTransacs(soup)
        i+=1
        if i ==5:
            break

SyntaxError: invalid syntax (<ipython-input-17-c08cdcab9943>, line 12)

In [10]:
conn.close()